In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [2]:
target_url = 'https://sfbay.craigslist.org/d/musical-instruments/search/msa?s='
url_prefix = 'https://sfbay.craigslist.org/d/musical-instruments/search/msa?s='

pages = ['120','240','360','480','600','720','840',
         '960','1080','1200','1320','1440','1560','1680',
         '1800','1920','2040','2160','2280','2400','2520',
         '2640','2760','2880','3000']
        
#test      
#pages = ['120', '240', '360']

url_list = []

for page in pages:
    url = url_prefix + page
    url_list.append(url)

In [3]:
url_list;

In [4]:
df_list = []
each_html_output = []

def price_pics_hood_title_datetime(starting_url):
    has_pics_bool = []
    price = []
    just_titles = []
    HOOD_list = []
    just_posted_datetimes = []
    
    for url in url_list:
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, 'html.parser')
        results = soup.find_all('li', class_='result-row')
        
    
    
#### Trying to get all the 5 features togther into 1 function.  
#### Maybe 1 cell.

        for res in results:
            """PRICE"""
            ## Loop for finding the PRICE of a single page of 120 results
            p = res.find('span', class_='result-price').text
            price.append(p)

            """PICS"""
            ## Loop for finding the boolean HAS PICS of a single page of 
            ## 120 results
            if res.find('span', class_='pictag') is None:
                has_pics_bool.append("False")
            else:
                has_pics_bool.append('True')


            """NEIGHBORHOOD"""
            ## Loop for finding NEIGHBORHOOD name for a single page of 120 results
            if res.find('span', class_="result-hood") is None:
                HOOD_list.append("NONE")
            else:    
                h = res.find('span', class_="result-hood").text
                HOOD_list.append(h)

        """TITLE"""    
        ## Loop for TITLE    
        titles=soup.find_all('a', class_="result-title hdrlnk")
        for title in titles:
            just_titles.append(title.text) 

        """DATETIME"""
        posted_datetimes=soup.find_all(class_='result-date')
        for posted_datetime in posted_datetimes:
            if posted_datetime.has_attr('datetime'):
                just_posted_datetimes.append(posted_datetime['datetime'])       
            
    comp_dict = {'price': price, 
                'pics': has_pics_bool,
                'hood': HOOD_list,
                'title': just_titles,
                'datetimes': just_posted_datetimes}

          
    return comp_dict

    print(len(price))
    print(len(has_pics_bool))
    print(len(HOOD_list))
    print(len(just_titles))
    print(len(just_posted_datetimes))

In [5]:
base_dict = price_pics_hood_title_datetime(target_url)
base_dict;

In [6]:
df_base = pd.DataFrame(base_dict)
df_base

,price,pics,hood,title,datetimes
0,$65,False,(hayward / castro valley),Amperex bugle boy mullard 5ar4 gz34,2021-01-19 18:45
1,$700,False,(hayward / castro valley),American standard fender precision bass wanted,2021-01-19 18:45
2,$75,True,(hayward / castro valley),Jbl 2118h 8” woofer/midrange,2021-01-19 18:45
3,$10,True,(hayward / castro valley),Crossovers 3 way,2021-01-19 18:45
4,$775,True,(hayward / castro valley),Fractal Axefx ultra. (Mint),2021-01-19 18:44
...,...,...,...,...,...
2875,$40,True,(marina / cow hollow),Vox VX12 Celestion G12P-80 16 ohm Guitar Speak...,2021-01-17 09:00
2876,$900,True,(oakland piedmont / montclair),King 2b+ Trombone,2021-01-17 09:00
2877,$250,True,(willow glen / cambrian),Line 6 Pod HD500 Multi-Effects Guitar Pedal,2021-01-17 08:59
2878,"$1,800",True,(novato),1921 Chickering Baby Grand Piano,2021-01-17 08:57


In [7]:
df_base.sort_values('datetimes')

,price,pics,hood,title,datetimes
2879,$50,True,(san jose east),Selmar Bundy flute with case vintage,2021-01-17 08:53
2878,"$1,800",True,(novato),1921 Chickering Baby Grand Piano,2021-01-17 08:57
2877,$250,True,(willow glen / cambrian),Line 6 Pod HD500 Multi-Effects Guitar Pedal,2021-01-17 08:59
2876,$900,True,(oakland piedmont / montclair),King 2b+ Trombone,2021-01-17 09:00
2875,$40,True,(marina / cow hollow),Vox VX12 Celestion G12P-80 16 ohm Guitar Speak...,2021-01-17 09:00
...,...,...,...,...,...
4,$775,True,(hayward / castro valley),Fractal Axefx ultra. (Mint),2021-01-19 18:44
3,$10,True,(hayward / castro valley),Crossovers 3 way,2021-01-19 18:45
2,$75,True,(hayward / castro valley),Jbl 2118h 8” woofer/midrange,2021-01-19 18:45
1,$700,False,(hayward / castro valley),American standard fender precision bass wanted,2021-01-19 18:45


In [13]:
df_base.to_csv('/Users/johnmetzger/Desktop/Coding/Project2/base_scrape.csv', index = False)